In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO


In [18]:
def get_complete_report(url):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all table elements
        tables = soup.find_all('table')
        
        # Initialize a counter for naming CSV files
        table_count = 0
        
        # Loop through each table
        for table in tables:
            # Attempt to find a caption for the table
            caption = table.find('caption')
            if caption is None:
                # If no caption is found, check for the closest preceding header (h1, h2, h3, etc.)
                for sibling in table.previous_siblings:
                    if sibling.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                        caption = sibling.text.strip()
                        break
            
            # Use caption text or a default name if caption is not found
            
            if caption:
                table_name = caption if isinstance(caption, str) else caption.text.strip()
                table_name = table_name.replace('/', ' or ')  # Replace problematic characters in file names
                if "Complete" in table_name:
            
                    # Convert the table to a DataFrame
                    df = pd.read_html(str(table))[0]
            table_count += 1
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
    return df


In [19]:


# URL of the page to scrape
url = 'https://fbref.com/en/players/7aa8adfe/scout/365_m1/Alejandro-Garnacho-Scouting-Report'

# Call the function with the specified URL
df = get_complete_report(url)


C:\Users\Steve\AppData\Local\Temp\ipykernel_14304\2758915922.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


In [21]:
df = df[~(df.isnull().all(axis=1))]

In [45]:
def all_elements_equal(row):
    return all(x == row.iloc[0] for x in row)

# Apply this function to each row
equal_rows = df[df.apply(all_elements_equal, axis=1)]

In [46]:
row = pd.DataFrame(data=[["Standard","Standard","Standard"]], columns=df.columns, index=[0])
equal_rows = pd.concat([equal_rows, row])
equal_rows = equal_rows.sort_index()
equal_rows.columns = [x[1] for x in equal_rows.columns]
equal_rows = equal_rows.Statistic

In [56]:
df.drop(index=equal_rows.index)

Standard Stats                               Category
                   Statistic Per 90 Percentile                     
1                    Assists   0.14         37             Standard
2            Goals + Assists   0.46         57             Standard
3          Non-Penalty Goals   0.32         76             Standard
4         Penalty Kicks Made   0.00         35             Standard
5    Penalty Kicks Attempted   0.00         32             Standard
..                       ...    ...        ...                  ...
163                Own Goals   0.00         51  Miscellaneous Stats
164          Ball Recoveries   4.29         47  Miscellaneous Stats
166              Aerials Won   0.25         20  Miscellaneous Stats
167             Aerials Lost   0.96         43  Miscellaneous Stats
168         % of Aerials Won  20.6%         11  Miscellaneous Stats

[141 rows x 4 columns]

In [50]:
range_dict = {}
prev_key = None
for key, value in equal_rows.items():
    if prev_key is not None:
        range_dict[(prev_key, key)] = equal_rows[prev_key]
    prev_key = key
# Assuming the DataFrame does not exceed the maximum index from series
range_dict[(prev_key, float('inf'))] = equal_rows[prev_key]

# Sample DataFrame 'df' - assuming it's already defined with some data and indices
# df = pd.DataFrame({'Data': [1, 2, 3, ..., n]}, index=[0, 1, 2, ..., m])

# Function to find the correct category based on index
def find_category(index):
    for (start, end), category in range_dict.items():
        if start <= index < end:
            return category
    return None  # in case no category matches

# Apply the function to the index of df to create a new column
df['Category'] = df.index.map(find_category)
